### A spelling checker

In 1975 Steve Johnson of Bell Labs wrote the first version of `spell` in an afternoon, which would later become the standard Unix spelling checker for the English language.

``` bash
prepare filename |                 # remove formatting commands
    translit A-Z a-z |             # map upper to lower case
        translit ^a-z @n |         # remove punctuation
            sort |                 # put words in alphabetical order
                unique |           # remove duplicate words
                    common -2 dict # report words not in dictionary
```
(from programming pearls Jon Bentley ...) `# TODO cite correctly`

In [35]:
from typing import Set
import re

Standard dictionary file is quite big, consisting of 99171 different words taking up 920K of space. This is not a major issue with modern computers that have memory in the gigabytes but it severely increases the runtime of the spelling check. `# TODO insert timings with different files`

In [54]:
def spell(filename: str, words = '/usr/share/dict/words') -> Set[str]:
    with open(filename) as f, open(words) as w:
        w = set(w.read().split('\n'))
        return set(sorted(re.sub(r"[^a-z]", "\n", f.read().lower()).split('\n'))) - w

Spell like spelling checker in just 4 lines of python (excl. imports) by utilizing the standard libraries.   
Obvious shortcomings like converting `isn't` to `isn` and `t` because punctuation is stripped.

In [59]:
spell('sample-text')

{'creepeth', 'doesn', 'isn', 'll', 'moveth'}

['A\n', "A's\n", "AA's\n", "AB's\n", "ABM's\n", "AC's\n", "ACTH's\n", "AI's\n", "AIDS's\n", "AM's\n", 'AOL\n', "AOL's\n", "ASCII's\n", "ASL's\n", "ATM's\n", "ATP's\n", "AWOL's\n", "AZ's\n", "AZT's\n", 'Aachen\n', 'Aaliyah\n', "Aaliyah's\n", 'Aaron\n', 'Abbas\n', 'Abbasid\n', 'Abbott\n', "Abbott's\n", 'Abby\n', "Abby's\n", 'Abdul\n', "Abdul's\n", 'Abe\n', "Abe's\n", 'Abel\n', "Abel's\n", 'Abelard\n', 'Abelson\n', "Abelson's\n", 'Aberdeen\n', "Aberdeen's\n", 'Abernathy\n', "Abernathy's\n", 'Abidjan\n', "Abidjan's\n", 'Abigail\n', 'Abilene\n', 'Abner\n', "Abner's\n", 'Abraham\n', "Abraham's\n", 'Abram\n', "Abram's\n", 'Abrams\n', 'Absalom\n', 'Abuja\n', 'Abyssinia\n', "Abyssinia's\n", 'Abyssinian\n', "Abyssinian's\n", 'Ac\n', "Ac's\n", 'Acadia\n', "Acadia's\n", 'Acapulco\n', "Acapulco's\n", 'Accenture\n', "Accenture's\n", 'Accra\n', "Accra's\n", 'Acevedo\n', "Acevedo's\n", 'Achaean\n', "Achaean's\n", 'Achebe\n', "Achebe's\n", 'Achernar\n', 'Acheson\n', "Acheson's\n", 'Achilles\n', "Achill

In [47]:
{'hi','there','wolrd'} - {'hit','there','world'}

{'hi', 'wolrd'}